In [2]:
import pandas as pd

## read the dataset

In [3]:
# Try loading JSON file without lines=True
df = pd.read_csv("C:\\Users\\tejag\\Downloads\\titanic.csv")
df.sample(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
41,1,1,"Brown, Mrs. James Joseph (Margaret Tobin)",female,44,0,0,PC 17610,27.7208,B4,C,6,?,"Denver, CO"
543,2,0,"Reeves, Mr. David",male,36,0,0,C.A. 17248,10.5,?,S,?,?,"Brighton, Sussex"
181,1,1,"Leader, Dr. Alice (Farnham)",female,49,0,0,17465,25.9292,D17,S,8,?,"New York, NY"
431,2,0,"Harper, Rev. John",male,28,0,1,248727,33,?,S,?,?,"Denmark Hill, Surrey / Chicago"
295,1,1,"Thayer, Mr. John Borland Jr",male,17,0,2,17421,110.8833,C70,C,B,?,"Haverford, PA"
1263,3,0,"van Billiard, Master. Walter John",male,11.5,1,1,A/5. 851,14.5,?,S,?,1,?
889,3,1,"Johansson Palmquist, Mr. Oskar Leander",male,26,0,0,347070,7.775,?,S,15,?,?
890,3,0,"Johansson, Mr. Erik",male,22,0,0,350052,7.7958,?,S,?,156,?
129,1,1,"Geiger, Miss. Amalie",female,35,0,0,113503,211.5,C130,C,4,?,?
353,2,1,"Bryhl, Miss. Dagmar Jenny Ingeborg",female,20,1,0,236853,26,?,S,12,?,"Skara, Sweden / Rockford, IL"


## Getting the Bing Liu lexicon

In [4]:
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\tejag\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


## Total number of words

In [5]:
print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))


Total number of words in opinion lexicon 6789


## Example of positive words

In [6]:
print('Examples of positive words in opinion lexicon',
opinion_lexicon.positive()[:10])


Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']


## Example of negative  words

In [7]:
print('Examples of negative words in opinion lexicon',
opinion_lexicon.negative()[:10])

Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


## Creating the Dictionary

In [17]:
#Let's create a dictionary which we can use for scoring our review text
nltk.download('punkt')
df.rename(columns={"reviewText": "home.dest"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tejag\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
    word_dict[word] = pos_score

In [19]:
# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
    word_dict[word] = neg_score

## Math Function

In [21]:
def bing_liu_score(text):
    sentiment_score = 0
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
    return sentiment_score

In [22]:
df['home.dest'].fillna('no review', inplace=True)
df['Bing_Liu_Score'] = df['home.dest'].apply(bing_liu_score)

## Output Code

In [24]:
df[['name',"home.dest", 'Bing_Liu_Score']].head(10)

,name,home.dest,Bing_Liu_Score
0,"Allen, Miss. Elisabeth Walton","St Louis, MO",0
1,"Allison, Master. Hudson Trevor","Montreal, PQ / Chesterville, ON",0
2,"Allison, Miss. Helen Loraine","Montreal, PQ / Chesterville, ON",0
3,"Allison, Mr. Hudson Joshua Creighton","Montreal, PQ / Chesterville, ON",0
4,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)","Montreal, PQ / Chesterville, ON",0
5,"Anderson, Mr. Harry","New York, NY",0
6,"Andrews, Miss. Kornelia Theodosia","Hudson, NY",0
7,"Andrews, Mr. Thomas Jr","Belfast, NI",0
8,"Appleton, Mrs. Edward Dale (Charlotte Lamson)","Bayside, Queens, NY",0
9,"Artagaveytia, Mr. Ramon","Montevideo, Uruguay",0


In [26]:
df.groupby('name').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
name,
"Abbing, Mr. Anthony",0.0
"Abbott, Master. Eugene Joseph",1.0
"Abbott, Mr. Rossmore Edward",1.0
"Abbott, Mrs. Stanton (Rosa Hunt)",1.0
"Abelseth, Miss. Karen Marie",0.0
...,...
"del Carlo, Mrs. Sebastiano (Argenia Genovesi)",0.0
"van Billiard, Master. James William",0.0
"van Billiard, Master. Walter John",0.0
